In [1]:
import azureml.core
print(azureml.core.VERSION)

1.0.53


In [4]:
from azureml.core import Workspace
ws = Workspace.create(name='ffdrone',
                      subscription_id='19a30e48-20e2-4ac5-b2fa-ac1ded374414',	
                      resource_group='firedrone-aml',
                      create_resource_group=True,
                      location='eastus2' 
                     )

Deploying AppInsights with name ffdroneinsights27d826128.
Deployed AppInsights with name ffdroneinsights27d826128. Took 3.36 seconds.
Deploying KeyVault with name ffdronekeyvaultb5c60fad8.
Deploying StorageAccount with name ffdronestorage48f04590d9.
Deployed KeyVault with name ffdronekeyvaultb5c60fad8. Took 17.49 seconds.
Deployed StorageAccount with name ffdronestorage48f04590d9. Took 24.14 seconds.
Deploying Workspace with name ffdrone.
Deployed Workspace with name ffdrone. Took 23.7 seconds.


In [6]:
ws.write_config()

In [8]:
ws = Workspace.from_config()

In [9]:
ws

Workspace.create(name='ffdrone', subscription_id='19a30e48-20e2-4ac5-b2fa-ac1ded374414', resource_group='firedrone-aml')

In [10]:
from azureml.core.model import Model

model = Model.register(model_path = "./models",
                        model_name = "firenet",
                        description = "detects fire in an image",
                        workspace = ws)

Registering model firenet


In [44]:
%%writefile score.py
import json
import numpy as np
import os
import sys
import tensorflow as tf

from azureml.core.model import Model

def init():
    tf.reset_default_graph()
    model_root = Model.get_model_path('firenet')
    model_path = os.path.join(model_root, 'firenet.pb')
    
    global graph, sess, X, output
    graph = tf.Graph()
    sess = tf.InteractiveSession(graph=graph)

    with tf.gfile.GFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    X = tf.placeholder(np.float32, shape=[None, 224, 224, 3], name='X')

    tf.import_graph_def(graph_def, {'InputData/X': X})
    output = graph.get_tensor_by_name("import/FullyConnected_2/Softmax:0")

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    out = sess.run(output, feed_dict={X: [data]})
    return out.tolist()

Overwriting score.py


In [45]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('numpy')
# make sure tensorflow is 1.14.0 version
cd.add_tensorflow_conda_package()
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults==1.0.53.*
- numpy
- tensorflow=1.13.1
channels:
- conda-forge



In [46]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=4, 
                                               tags={'name':'ffdrone', 'framework': 'TensorFlow DNN'},
                                               description='Tensorflow DNN on MNIST')


In [47]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [48]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='ffdrone-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

Creating image
Running....................................................
Succeeded
Image creation operation finished for image ffdrone-svc:3, operation "Succeeded"
Creating service
Running..............................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 1.13 s, sys: 191 ms, total: 1.32 s
Wall time: 7min 6s


In [49]:
print(service.scoring_uri)

http://dbec25b5-842a-4fe6-a484-6b252cbb37e9.eastus2.azurecontainer.io/score


In [54]:
import json
import cv2

rows = 224
cols = 224
img = cv2.imread("../firedrone/frame.png" )
small_frame = cv2.resize(img, (rows, cols), cv2.INTER_AREA)
test = json.dumps({"data": small_frame.tolist()})
test = bytes(test, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test)
print(result)

[[0.9982970356941223, 0.0017029689624905586]]


In [55]:
service.delete()